**Capstone Project Submission**

* Student Name: Wes Swager
* Student Pace: Full Time
* Instructor Name: Claude Fried
* Scheduled Project Review Date/Time
    * Friday, June 11, 2021, 2:30pm CST
    * Monday, June 13, 2021,

# **Expected Goals Classifier**

## Overview

Create an expected goals classification model using existing historical match data for use with future match analysis and actionable recommendations which can be utilized in training to help improve goal-scoring.

Project detailed on Github: [milwaukee_fc](https://github.com/wswager/milwaukee_fc)

# Feature Engineering Notebook

*Notebook 3 of 8*

### Index

1. Data extracted in [expected_goals_data_extraction_notebook]()
2. Data organized in [expected_goals_data_organization_notebook]()
3. Features engineered in [expected_goals_feature_engineering_notebook]()
4. Data cleaned in [expected_goals_data_cleaning_notebook]()
5. Data explored in [expected_goals_data_exploration_notebook]()
6. Data preprocessed in [expected_goals_data_preprocessing_notebook]()
7. Model fitting and refinement in [expected_goals_model_fitting_notebook]()
8. Model assessment in [expected_goals_model_assessment_notebook]()

### Data

Data sourced from [StatsBomb](https://statsbomb.com/), a United Kingdom based football (soccer) data analytics company.

StatsBomb have provided free access to their proprietary dataset via GitHub: [StatsBomb Open Data](https://github.com/statsbomb/open-data)

In [59]:
# Import organized_data from expected_goals_data_organization_notebook

organized_data = pd.read_csv('/content/drive/MyDrive/flatiron/expected_goals/data_organization/organized_data.csv')

In [60]:
organized_data = organized_data.iloc[: , 1:]

In [61]:
organized_data.head()

,location_x,location_y,time,statsbomb_xg,outcome,player,team,bodypart,technique,first_touch,state_of_play,assist1,assist2,assist3,assist_state_of_play,inside_18_width,inside_18_depth,inside_18,shot_distance,shot_angle
0,109.0,46.0,2021-06-14 00:04:38.609,0.266154,Blocked,Francesca Kirby,Chelsea FCW,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,Regular Play,True,True,True,12.529964,118.61
1,113.0,35.0,2021-06-14 00:11:45.046,0.093521,Off T,Bethany England,Chelsea FCW,Head,Normal,False,Open Play,High Pass,NaN,NaN,From Free Kick,True,True,True,8.602325,54.46
2,94.0,43.0,2021-06-14 00:18:03.461,0.036171,Saved,Drew Spence,Chelsea FCW,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,Regular Play,True,False,False,26.172505,96.58
3,86.0,34.0,2021-06-14 00:23:11.935,0.016625,Off T,Chloe Arthur,Birmingham City WFC,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,From Goal Kick,True,False,False,34.525353,79.99
4,94.0,33.0,2021-06-14 00:23:45.810,0.030716,Off T,Bethany England,Chelsea FCW,Right Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,From Goal Kick,True,False,False,26.925824,74.93


# Packages

In [1]:
# Drive  and IO to access saved data
from google.colab import drive, files
drive.mount('/content/drive')

import io

# Pathlib for file retrieval
import pathlib
from pathlib import Path

# Pandas for Dataframes
import pandas as pd

# Numpy for mathematical functions
import numpy as np

import math
from math import atan2

# Shapely for geometric functions
import shapely
from shapely import wkt
from shapely.geometry import Point, Polygon, LineString, GeometryCollection

import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


# Inside 18-Yard Box

### Width

In [62]:
# Calculate if the shot was taken within the width of the 18-yard box

inside_18_width_list = []
for i in range(0, len(organized_data)):
  if (organized_data.iloc[i]['location_y'] > 22) & (organized_data.iloc[i]['location_y'] < 58):
    inside_18_width_list.append(True)
  
  else:
    inside_18_width_list.append(False)

organized_data['inside_18_width'] = inside_18_width_list

In [63]:
organized_data['inside_18_width'].value_counts()

True     5719
False     385
Name: inside_18_width, dtype: int64

### Depth

In [64]:
# Calculate if the shot was taken within the depth of the 18-yard box

inside_18_depth_list = []
for i in range(0, len(organized_data)):
  if (organized_data.iloc[i]['location_x'] > 102):
    inside_18_depth_list.append(True)
  
  else:
    inside_18_depth_list.append(False)

organized_data['inside_18_depth'] = inside_18_depth_list

In [65]:
organized_data['inside_18_depth'].value_counts()

True     3747
False    2357
Name: inside_18_depth, dtype: int64

### Total

In [66]:
# Calculate if the shot was taken within the 18-yard box

inside_18_list = []
for i in range(0, len(organized_data)):
  if ((organized_data.iloc[i]['inside_18_width'] == True) &
      (organized_data.iloc[i]['inside_18_depth'] == True)):
    inside_18_list.append(True)
  
  else:
    inside_18_list.append(False)

organized_data['inside_18'] = inside_18_list

In [67]:
organized_data['inside_18'].value_counts()

True     3588
False    2516
Name: inside_18, dtype: int64

# Distance

In [68]:
# Define goal center
# Field coordinates for events measured for team in-possession
# Goal center will be consistent for both home and away teams
# because map in flipped and consistent for both teams

goal_center = (120, 40)

In [69]:
# Use location_x and location_y to define shot coordinates

shot_location_list = []
for i in range(0, len(organized_data)):
  shot_location_list.append((organized_data.iloc[i]['location_x'],
                             organized_data.iloc[i]['location_y']))

In [70]:
# Calculate distance from shot location to goal_center

shot_distance_list = []
for sl in shot_location_list:
  shot_distance_list.append(Point(sl).distance(Point(goal_center)))

organized_data['shot_distance'] = shot_distance_list

In [71]:
pd.DataFrame(organized_data['shot_distance'].describe())

,shot_distance
count,6104.000000
mean,18.916359
std,9.071373
min,1.000000
25%,11.526057
50%,17.804494
75%,25.546526
max,66.540213


# Angle

In [72]:
# Calculate angle between the shot location and goal_center

shot_angle_list = []
for i in range(0, len(organized_data)):
  shot_angle_list.append(round(math.degrees(math.atan2((goal_center[0] - organized_data.iloc[i]['location_x']),
                                                       (goal_center[1] - organized_data.iloc[i]['location_y']))), 2))

organized_data['shot_angle'] = shot_angle_list

In [73]:
pd.DataFrame(organized_data['shot_angle'].describe())

,shot_angle
count,6104.000000
mean,91.022638
std,33.911162
min,0.000000
25%,64.650000
50%,90.460000
75%,116.995000
max,180.000000


# Bodypart Angle

In [74]:
organized_data['bodypart'].value_counts()

Right Foot    3493
Left Foot     1676
Head           926
Other            9
Name: bodypart, dtype: int64

In [75]:
# Combare the side the shot was taken from to
# the foot the shot was taken with

bodypart_angle_list = []
for i in range(0, len(organized_data)):
  if ((organized_data.iloc[i]['shot_angle'] > 90) &
      (organized_data.iloc[i]['bodypart'] == 'Right Foot')):
    bodypart_angle_list.append('Right - Outside Foot')
  
  elif ((organized_data.iloc[i]['shot_angle'] > 90) &
        (organized_data.iloc[i]['bodypart'] == 'Left Foot')):
    bodypart_angle_list.append('Right - Inside Foot')
  
  elif ((organized_data.iloc[i]['shot_angle'] > 90) &
        (organized_data.iloc[i]['bodypart'] == 'Head')):
    bodypart_angle_list.append('Right - Head')
  
  elif ((organized_data.iloc[i]['shot_angle'] < 90) &
        (organized_data.iloc[i]['bodypart'] == 'Left Foot')):
    bodypart_angle_list.append('Left - Outside Foot')
  
  elif ((organized_data.iloc[i]['shot_angle'] < 90) &
        (organized_data.iloc[i]['bodypart'] == 'Right Foot')):
    bodypart_angle_list.append('Left - Inside Foot')
  
  elif ((organized_data.iloc[i]['shot_angle'] < 90) &
        (organized_data.iloc[i]['bodypart'] == 'Head')):
    bodypart_angle_list.append('Left - Head')
  
  else:
    bodypart_angle_list.append('Other')

organized_data['bodypart_angle'] = bodypart_angle_list

In [77]:
organized_data['bodypart_angle'].value_counts()

Right - Outside Foot    1882
Left - Inside Foot      1518
Left - Outside Foot      891
Right - Inside Foot      734
Right - Head             440
Left - Head              436
Other                    203
Name: bodypart_angle, dtype: int64

# Significant Time

In [78]:
# Convert time datatype to datetime

organized_data['time'] = organized_data['time'].astype(str)
organized_data['time'] = pd.to_datetime(organized_data['time'])

In [79]:
significant_time_list = []
for i in range(0, len(organized_data)):
  if organized_data.iloc[i]['time'] < pd.Timestamp(2021, 6, 14, 0, 5, 0):
    significant_time_list.append('First 5min')
  
  elif organized_data.iloc[i]['time'] >  pd.Timestamp(2021, 6, 14, 0, 45, 0):
    significant_time_list.append('Stoppage Time')
  
  elif organized_data.iloc[i]['time'] > pd.Timestamp(2021, 6, 14, 0, 40, 0):
    significant_time_list.append('Last 5min')
  
  else:
    significant_time_list.append('Regular Time')

organized_data['significant_time'] = significant_time_list

In [80]:
organized_data['significant_time'].value_counts()

Regular Time     4433
Last 5min         611
First 5min        611
Stoppage Time     449
Name: significant_time, dtype: int64

# Data with Engineered Features

In [81]:
data_with_engineered_features = organized_data
data_with_engineered_features.head()

,location_x,location_y,time,statsbomb_xg,outcome,player,team,bodypart,technique,first_touch,state_of_play,assist1,assist2,assist3,assist_state_of_play,inside_18_width,inside_18_depth,inside_18,shot_distance,shot_angle,bodypart_angle,significant_time
0,109.0,46.0,2021-06-14 00:04:38.609,0.266154,Blocked,Francesca Kirby,Chelsea FCW,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,Regular Play,True,True,True,12.529964,118.61,Right - Inside Foot,First 5min
1,113.0,35.0,2021-06-14 00:11:45.046,0.093521,Off T,Bethany England,Chelsea FCW,Head,Normal,False,Open Play,High Pass,NaN,NaN,From Free Kick,True,True,True,8.602325,54.46,Left - Head,Regular Time
2,94.0,43.0,2021-06-14 00:18:03.461,0.036171,Saved,Drew Spence,Chelsea FCW,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,Regular Play,True,False,False,26.172505,96.58,Right - Inside Foot,Regular Time
3,86.0,34.0,2021-06-14 00:23:11.935,0.016625,Off T,Chloe Arthur,Birmingham City WFC,Left Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,From Goal Kick,True,False,False,34.525353,79.99,Left - Outside Foot,Regular Time
4,94.0,33.0,2021-06-14 00:23:45.810,0.030716,Off T,Bethany England,Chelsea FCW,Right Foot,Normal,False,Open Play,Ground Pass,NaN,NaN,From Goal Kick,True,False,False,26.925824,74.93,Left - Inside Foot,Regular Time


In [82]:
data_with_engineered_features.to_csv('/content/drive/MyDrive/flatiron/expected_goals/feature_engineering/data_with_engineered_features.csv')

Continued in [expected_goals_data_cleaning_notebook]()